# Tensorboard 可视化训练

练丹炉子里的情况用print去看会很伤眼睛。Tensorflow有tensorboard模块支持训练结果的定制化可视化。

## 启动tensorboard

```bash
tensorboard --logdir logs
```

在浏览器里可以通过url和默认打开的6007端口进入tensorboard的UI.这里使用了本机所以是localhost，用其他机器的话确认url和主机hostname一致即可。

```url
http://localhost:6007/#
```

## 代码中创建tensorboard的接口

In [ ]:
# 用当前时间戳创建log的子目录
import datetime

current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = 'logs/' + current_time


In [ ]:
# 创建tensorboard的log的写句柄
import tensorflow as tf

log_writer = tf.summary.create_file_writer(log_dir)


## 给tensorboard喂数据

先把02里的代码copy&past过来

In [ ]:
from tensorflow import keras
from tensorflow.keras import datasets, layers, optimizers, Sequential, metrics

def preprocess(x, y):
    x = tf.cast(x, dtype=tf.float32) / 255. # cast is ok; but convert_to_tensor is not working-> uint8 can't be converted to float32 tensor
    y = tf.cast(y, dtype=tf.int64)
    return x, y

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data(path='./mnist.pnz')

train_db = tf.data.Dataset.from_tensor_slices((x_train, y_train)).map(preprocess).shuffle(10000).batch(100)

test_db = tf.data.Dataset.from_tensor_slices((x_test, y_test)).map(preprocess).batch(100)

network = keras.Sequential([
    layers.Dense(256, activation=tf.nn.relu),
    layers.Dense(128, activation=tf.nn.relu),
    layers.Dense(64, activation=tf.nn.relu),
    layers.Dense(32, activation=tf.nn.relu),
    layers.Dense(10)
])
network.build(input_shape=[None, 28*28]) # 给定输入层的结构从而初始化网络
network.summary() # 打印出网络结构

optimizer = optimizers.Adam(lr=1e-3) # 参数更新优化器，LR learnning rate

def main():
    for epoch in range(50):
        # Training...
        for step, (x, y) in enumerate(train_db):
            x = tf.reshape(x, [-1, 28*28])
            with tf.GradientTape() as tape:
                logits = network(x)
                y_onehot = tf.one_hot(y, depth=10)
                loss = tf.losses.categorical_crossentropy(y_onehot, logits, from_logits=True)

            grads = tape.gradient(loss, network.trainable_variables)
            optimizer.apply_gradients(zip(grads, network.trainable_variables))

            # if step % 100 == 0 : 
            #    print(epoch, step, 'loss:', float(tf.reduce_mean(loss)))
            # 这里用 tensorboard 取代 print
            if step % 100 == 0 :
                with log_writer.as_default():
                    tf.summary.scalar('loss', float(tf.reduce_mean(loss)), step=epoch)

        # Testing...
        total_correct, total_num = 0, 0
        for step, (x, y) in enumerate(test_db):
            x = tf.reshape(x, [-1, 28*28])
            out = network(x)

            prob = tf.nn.softmax(out, axis=1)
            pred = tf.argmax(prob, axis=1)
            
            correct = tf.cast(tf.equal(pred, y), dtype=tf.int32)
            correct = tf.reduce_sum(correct)
            
            total_correct += int(correct)
            total_num += x.shape[0]

        # print('test acc:', total_correct * 100 / total_num, '%')
        # 这里用 tensorboard 取代 print
        with log_writer.as_default():
            tf.summary.scalar('accuracy', float(total_correct/total_num), step=epoch)


if __name__ == '__main__':
    main()